<a href="https://colab.research.google.com/github/RomulusGwelt/AngularProject/blob/master/Hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading data 

Note: You need to upload kaggle API token for the following cell





In [0]:
#@title Default title text
# !unzip test.zip
# !unzip train.zip

# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json
# Next, install the Kaggle API client.
!pip install -q kaggle

# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

#download the dataset for the dog-breed identification challenge https://www.kaggle.com/c/dog-breed-identification
!kaggle competitions download -c dog-breed-identification

#unzip the downloaded files
!unzip test.zip
!unzip train.zip


In [0]:

# !pip uninstall tensorflow
# !pip install tensorflow-gpu

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from keras_preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

import numpy as np 
import pandas as pd 

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
import keras
from math import ceil
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.inception_v3 import preprocess_input
# from keras.applications.resnet50 import preprocess_input
from keras.applications.xception import preprocess_input
import os 
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras.callbacks import EarlyStopping, ModelCheckpoint
from cyclicLR import CyclicLR
from keras.optimizers import Adam, SGD, RMSprop
import datetime
from keras.applications import xception
# print(device_lib.list_local_devices())
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

CyclicLR can be found at my google drive or https://github.com/bckenstler/CLR

In [0]:
print(tf.__version__)

1.15.0


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# file_list = drive.ListFile({'q': "'16WevdpbCzQCiuKAeI3KP94BVzgGtcOox' in parents and trashed=false"}).GetList()
# file_list = drive.ListFile({'q': "'1s6iF8UJCONyJrkArmIUbOj5QRDO39vbg' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#   print('title: %s, id: %s' % (file1['title'], file1['id']))

In [0]:
link = 'https://drive.google.com/open?id=1XdEL-x4jM5ZeYPzpWQjWQtbh2ydMKen8'
fluff, id = link.split('=')
print (id)

1XdEL-x4jM5ZeYPzpWQjWQtbh2ydMKen8


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('labels.csv')  
ds = pd.read_csv('labels.csv')
ds['image'] = ds.apply(lambda row: str(row['id']) + ".jpg", axis=1)
ds = ds.sample(frac=1).reset_index(drop=True)
ds.head(10)

print(ds.head())


                                 id  ...                                 image
0  5811c9b5aa472c3596347c4467d9acd6  ...  5811c9b5aa472c3596347c4467d9acd6.jpg
1  b58abe0f8254c308bb3334e5784caf91  ...  b58abe0f8254c308bb3334e5784caf91.jpg
2  4a48d6eef7b4ae5e9eaed28d5b1bba73  ...  4a48d6eef7b4ae5e9eaed28d5b1bba73.jpg
3  322068ec9053585edd2ccc29228b08e8  ...  322068ec9053585edd2ccc29228b08e8.jpg
4  20d7b779380e37618be8a4e608f160e3  ...  20d7b779380e37618be8a4e608f160e3.jpg

[5 rows x 3 columns]


In [0]:
batch_size = 16

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

image_generator = ImageDataGenerator(
    # rescale=1./255,
    preprocessing_function=preprocess_input,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1
)

training_generator = image_generator.flow_from_dataframe(
    dataframe=ds,
    directory="train",
    # directory="/content/gdrive/My Drive/Colab Notebooks/train",
    x_col="image",
    y_col="breed",
    target_size=(299,299),
    batch_size=batch_size,
    subset="training"
)

validation_generator = image_generator.flow_from_dataframe(
    dataframe=ds,
    directory="train",
    # directory="/content/gdrive/My Drive/Colab Notebooks/train",
    x_col="image",
    y_col="breed",
    target_size=(299,299),
    batch_size=batch_size,
    subset="validation"
)

Found 9200 validated image filenames belonging to 120 classes.
Found 1022 validated image filenames belonging to 120 classes.


In [0]:
input_shape = (299, 299, 3)

classes = len(training_generator.class_indices)

In [0]:
# base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
# base_model = ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')
# base_model = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
base_model = xception.Xception(input_shape=input_shape, weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
# checkpoints
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
STAMP = "{}_dog_breed_model".format(datetime.date.today().strftime("%Y-%m-%d"))

bst_model_path = "models/{}.h5".format(STAMP)
model_checkpoint = ModelCheckpoint(bst_model_path,
                                   save_best_only=True,
                                   save_weights_only=True)

# Authors suggest setting step_size = (2-8) x (training iterations in epoch)
step_size = 2000
clr = CyclicLR(base_lr=0.0001,
               max_lr=0.001,
               step_size=step_size,
               mode='exp_range',
               gamma=0.99994)

In [0]:
for layer in base_model.layers:
    layer.trainable = False

optimizer = RMSprop(lr=0.001, rho=0.9)
# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

model.compile(optimizer=optimizer,
              # loss='sparse_categorical_crossentropy',
              loss = keras.losses.categorical_crossentropy,
              metrics=["accuracy"])

In [0]:
hist = model.fit_generator(
                           generator=training_generator,
                           steps_per_epoch=ceil(0.8 * (ds.size / batch_size)),
                           
                           validation_data=validation_generator,
                           validation_steps=ceil(0.2 * (ds.size / batch_size)),
                           
                           epochs=50, 
                           callbacks=[early_stopping, model_checkpoint, clr],
)
# model.fit_generator(
#     generator=training_generator,
#     steps_per_epoch=ceil(0.8 * (ds.size / batch_size)),

#     validation_data=validation_generator,
#     validation_steps=ceil(0.2 * (ds.size / batch_size)),

#     epochs=50,
#     verbose=1
# ) 

Epoch 1/50
1534/1534 [==============================] - 581s 379ms/step - loss: 1.1901 - acc: 0.6852 - val_loss: 0.7012 - val_acc: 0.7934
Epoch 2/50
1534/1534 [==============================] - 576s 376ms/step - loss: 0.9570 - acc: 0.7377 - val_loss: 0.6833 - val_acc: 0.7988
Epoch 3/50
1534/1534 [==============================] - 573s 373ms/step - loss: 1.0452 - acc: 0.7271 - val_loss: 0.7217 - val_acc: 0.7973
Epoch 4/50
1534/1534 [==============================] - 581s 379ms/step - loss: 0.9975 - acc: 0.7368 - val_loss: 0.6611 - val_acc: 0.8072
Epoch 5/50
1534/1534 [==============================] - 582s 379ms/step - loss: 0.9213 - acc: 0.7543 - val_loss: 0.6951 - val_acc: 0.8071
Epoch 6/50
1534/1534 [==============================] - 586s 382ms/step - loss: 1.0163 - acc: 0.7430 - val_loss: 0.6667 - val_acc: 0.8084
Epoch 7/50
1534/1534 [==============================] - 577s 376ms/step - loss: 0.9185 - acc: 0.7560 - val_loss: 0.6709 - val_acc: 0.8107
Epoch 8/50
1534/1534 [============

In [0]:
model.save('model.h5')


In [0]:
model.save_weights('model_weights.h5')

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
uploaded = drive.CreateFile({'title': 'model.h5'})
uploaded.SetContentFile('model.h5')
uploaded.Upload()
# uploaded.Download()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1Cf_8N1d2O82dTK47zqef3fjC9CDOXmVg


In [0]:
uploaded = drive.CreateFile({'title': 'model_weights.h5'})
uploaded.SetContentFile('model_weights.h5')
uploaded.Upload()
# uploaded.Download()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1j8cj4hc0ea1ljlBhjaNVKZbpzIcdmUHZ
